# Loads workitems from a query
A notebook to load data from devops, merge and clean the data before storing it to a file for analyzis

In [26]:
import os
import datetime
from pytz import timezone
import pandas as pd
from azure.devops.credentials import BasicAuthentication
from azure.devops.connection import Connection
from azure.devops.v7_1.work_item_tracking.models import Wiql
from enum import Enum

## Configuration 
auth_token is a personal access token (PAT) stored in a Windows environment variable called AZURE_DEVOPS_PAT

In [48]:
__VERSION__ = "1.0.0"
auth_token = os.environ['AZURE_DEVOPS_PAT']
url ="https://dev.azure.com/skanskanordic/"
area = "Skanska Sverige IT"
collect_period = "Skanska Sverige IT\\2023"
increment = "Skanska Sverige IT\\2023\Höst 2023-3"
date_of = datetime.datetime(2023,11, 9, tzinfo=timezone('UTC'))

In [49]:
from types import SimpleNamespace
context = SimpleNamespace()
context.runner_cache = SimpleNamespace()
context.connection = Connection(base_url=url,creds=BasicAuthentication('PAT', auth_token), user_agent='azure-devops-python-samples/' + __VERSION__)

In [50]:
wit_client = context.connection.clients.get_work_item_tracking_client()

## Workitems is loaded from devops using a query

In [51]:
wiql = Wiql(query="SELECT [System.Id] from WorkItems WHERE [System.AreaPath] under '"+area+"' AND [System.IterationPath] under '"+collect_period+'\\'+"' AND [System.WorkItemType] = 'Requirement'")
wiql_results = wit_client.query_by_wiql(wiql, top=10000).work_items

In [52]:
desired_ids = list()
for item in wiql_results: desired_ids.append(int(item.id))
len(desired_ids)

1216

# These items is then loaded from different time 

In [53]:
workitems_after_planning = list()
start = 0 
stop = 0
start
max = len(desired_ids)-1
print(str(max))
while stop<max:
    if (start+100)<=max:
        stop = start+99  
    else:
        stop = stop+(len(desired_ids) - start)

    workitems_after_planning += wit_client.get_work_items(ids=desired_ids[start:stop], as_of=date_of, error_policy="omit" )
    start+=100

1215


In [54]:
len(workitems_after_planning)

1203

In [55]:
def getTasks(workitems):
    result = list()
    for item in workitems:
        if (item is None): continue
        result.append(
            {
                "id": item.id,
                "area": item.fields["System.AreaPath"],#.split('\\')[-1],
                "iteration": item.fields["System.IterationPath"].split('\\')[-1],
                "title": item.fields["System.Title"],
                "state.start": item.fields["System.State"],
                "state.end": item.fields["System.State"],
                "resource": item.fields["System.AssignedTo"]["displayName"] if "System.AssignedTo" in item.fields else "", 
                "estimate": item.fields["Microsoft.VSTS.Scheduling.OriginalEstimate"] if "Microsoft.VSTS.Scheduling.OriginalEstimate" in item.fields else "", 
                "completed": item.fields["Microsoft.VSTS.Scheduling.CompletedWork"] if "Microsoft.VSTS.Scheduling.CompletedWork" in item.fields else "",
                "remaining" : item.fields["Microsoft.VSTS.Scheduling.RemainingWork"] if "Microsoft.VSTS.Scheduling.RemainingWork" in item.fields else "",
                "activated" : item.fields["Microsoft.VSTS.Common.ActivatedDate"] if "Microsoft.VSTS.Common.ActivatedDate" in item.fields else "",
                "resolved": item.fields["Microsoft.VSTS.Common.ResolvedDate"] if "Microsoft.VSTS.Common.ResolvedDate" in item.fields else ""
            })  
    return result

In [56]:
tasks_after_planning = pd.DataFrame(getTasks(workitems_after_planning))


In [57]:
#tasks_after_planning.info()
tasks_after_planning

,id,area,iteration,title,state.start,state.end,resource,estimate,completed,remaining,activated,resolved
0,45871,Skanska Sverige IT\Development and Operations\...,2023,Returhantering på platta,Proposed,Proposed,,,,,,
1,49806,Skanska Sverige IT\Development and Operations\...,Sommar 2023-2,Maskinindividnummer vid service ska med på fak...,Closed,Closed,Ronnie Johansson,40.0,,,2023-06-30T10:16:44.743Z,2023-07-04T12:26:11.597Z
2,53664,Skanska Sverige IT\Development and Operations\...,2023,Plockorder i platta/padda,Proposed,Proposed,Eva Walther,60.0,,,,
3,54035,Skanska Sverige IT\Development and Operations\...,Höst 2023-3,Rental Webbshop - Nytt/förbättrat sätt att log...,Active,Active,Lars Marmestedt,40.0,,,2022-09-08T12:29:00.567Z,
4,54052,Skanska Sverige IT\Common Service\Digital arbe...,2023,Telefoni - SWAP service assetportal,Ready,Ready,Petra Mård,80.0,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
1198,94919,Skanska Sverige IT\Development and Operations\...,Vinter 2023-4,"Create ""opti"" UI Elements NPM package",Active,Active,"Leo Lindeberg, Dan",20.0,,,2023-11-08T10:13:22.793Z,
1199,94929,Skanska Sverige IT\Development and Operations\...,Vinter 2023-4,Foresight: Migrate content to CoreExperience i...,Active,Active,Susanne Holmberg,6.0,,,2023-11-08T09:35:43.92Z,
1200,94930,Skanska Sverige IT\Development and Operations\...,Vinter 2023-4,ValueMoment: Migrate content to CoreExperience...,Active,Active,Susanne Holmberg,6.0,,,2023-11-08T09:35:43.92Z,
1201,94934,Skanska Sverige IT\Development and Operations\...,Vinter 2023-4,Bostad Web - Maintenance (L4-2023),Active,Active,"Ring, Ann-Margret",50.0,,,2023-11-08T10:02:50.97Z,


In [58]:
df = tasks_after_planning.replace("NaN", 0)
df['estimate'].replace('',0)
df['completed'].replace('',0)
df['remaining'].replace('',0)
df['estimate'] = pd.to_numeric(df['estimate'])
df['completed'] = pd.to_numeric(df['completed'])
df['remaining'] = pd.to_numeric(df['remaining'])
#df.info()
df

,id,area,iteration,title,state.start,state.end,resource,estimate,completed,remaining,activated,resolved
0,45871,Skanska Sverige IT\Development and Operations\...,2023,Returhantering på platta,Proposed,Proposed,,NaN,NaN,NaN,,
1,49806,Skanska Sverige IT\Development and Operations\...,Sommar 2023-2,Maskinindividnummer vid service ska med på fak...,Closed,Closed,Ronnie Johansson,40.0,NaN,NaN,2023-06-30T10:16:44.743Z,2023-07-04T12:26:11.597Z
2,53664,Skanska Sverige IT\Development and Operations\...,2023,Plockorder i platta/padda,Proposed,Proposed,Eva Walther,60.0,NaN,NaN,,
3,54035,Skanska Sverige IT\Development and Operations\...,Höst 2023-3,Rental Webbshop - Nytt/förbättrat sätt att log...,Active,Active,Lars Marmestedt,40.0,NaN,NaN,2022-09-08T12:29:00.567Z,
4,54052,Skanska Sverige IT\Common Service\Digital arbe...,2023,Telefoni - SWAP service assetportal,Ready,Ready,Petra Mård,80.0,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...
1198,94919,Skanska Sverige IT\Development and Operations\...,Vinter 2023-4,"Create ""opti"" UI Elements NPM package",Active,Active,"Leo Lindeberg, Dan",20.0,NaN,NaN,2023-11-08T10:13:22.793Z,
1199,94929,Skanska Sverige IT\Development and Operations\...,Vinter 2023-4,Foresight: Migrate content to CoreExperience i...,Active,Active,Susanne Holmberg,6.0,NaN,NaN,2023-11-08T09:35:43.92Z,
1200,94930,Skanska Sverige IT\Development and Operations\...,Vinter 2023-4,ValueMoment: Migrate content to CoreExperience...,Active,Active,Susanne Holmberg,6.0,NaN,NaN,2023-11-08T09:35:43.92Z,
1201,94934,Skanska Sverige IT\Development and Operations\...,Vinter 2023-4,Bostad Web - Maintenance (L4-2023),Active,Active,"Ring, Ann-Margret",50.0,NaN,NaN,2023-11-08T10:02:50.97Z,


# The dataframe is stored as csv into a file

In [46]:
df.to_csv("requirements.csv", sep=";", decimal=',')